In [30]:
# Data Preprocessing
# Author: Shabd Mishra

import pandas as pd
import numpy as np

#Load Excel file and read the sheet
df = pd.read_excel(
    "Canada.xlsx",
    sheet_name="Canada by Citizenship",
    skiprows=range(20)
)

# Ensure the column names are strings 
df.columns = df.columns.map(str)

# Normalize column names to clean strings
df.columns = [
    c.replace(".0", "") if isinstance(c, str) and c.endswith(".0") else c
    for c in df.columns
]


# Find all year columns
year_cols = []
for c in df.columns:
    c_clean = c.replace(".0", "")       
    if c_clean.isdigit() and len(c_clean) == 4:
        year_cols.append(c)


# Sort years properly
year_cols = sorted(year_cols, key=lambda x: int(x.replace(".0", "")))

# 3) Convert year values to integers
df[year_cols] = df[year_cols].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)

# Filter dataset to meet the requirements
df = df[(df["Type"] == "Immigrants") & (df["Coverage"] == "Foreigners")].copy()
df = df[~df["OdName"].isin(["Total", "Unknown"])].copy()

# Final cleaned dataset
final_df = df.rename(columns={
    "OdName": "Country",
    "AreaName": "Continent",
    "RegName": "Region"
})[["Country", "Continent", "Region", "DevName"] + year_cols].copy()

final_df["Total"] = final_df[year_cols].sum(axis=1)

def ycol(y):
    # Find the matching column name in year_cols for a year ("1980" vs "1980.0")
    for c in year_cols:
        if int(c.replace(".0", "")) == y:
            return c
    raise KeyError(f"Year {y} not found")

# Select early and late year ranges
early = [ycol(y) for y in range(1980, 1987)]
late  = [ycol(y) for y in range(2005, 2014)]

# Build the final display DataFrame
display_df = final_df[["Country", "Continent", "Region", "DevName"] + early].copy()
display_df["..."] = "..."
display_df = pd.concat([display_df, final_df[late + ["Total"]]], axis=1)

# Display the results (Country as index)
display_df.set_index("Country").head()


,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,1986,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,496,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,1,...,1223,856,702,560,716,561,539,620,603,15699
Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,69,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439
American Samoa,Oceania,Polynesia,Developing regions,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,6
Andorra,Europe,Southern Europe,Developed regions,0,0,0,0,0,0,2,...,0,1,1,0,0,0,0,1,1,15


In [25]:
display_df.tail()

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
190,Viet Nam,Asia,South-Eastern Asia,Developing regions,1191,1829,2162,3404,7583,5907,...,1852,3153,2574,1784,2171,1942,1723,1731,2112,97146
191,Western Sahara,Africa,Northern Africa,Developing regions,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
192,Yemen,Asia,Western Asia,Developing regions,1,2,1,6,0,18,...,161,140,122,133,128,211,160,174,217,2985
193,Zambia,Africa,Eastern Africa,Developing regions,11,17,11,7,16,9,...,91,77,71,64,60,102,69,46,59,1677
194,Zimbabwe,Africa,Eastern Africa,Developing regions,72,114,102,44,32,29,...,615,454,663,611,508,494,434,437,407,8598


In [26]:
display_df.head(20)

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
0,Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
1,Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,...,1223,856,702,560,716,561,539,620,603,15699
2,Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439
3,American Samoa,Oceania,Polynesia,Developing regions,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,6
4,Andorra,Europe,Southern Europe,Developed regions,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,15
5,Angola,Africa,Middle Africa,Developing regions,1,3,6,6,4,3,...,295,184,106,76,62,61,39,70,45,2113
6,Antigua and Barbuda,Latin America and the Caribbean,Caribbean,Developing regions,0,0,0,0,42,52,...,24,32,15,32,38,27,37,51,25,981
7,Argentina,Latin America and the Caribbean,South America,Developing regions,368,426,626,241,237,196,...,1153,847,620,540,467,459,278,263,282,19596
8,Armenia,Asia,Western Asia,Developing regions,0,0,0,0,0,0,...,224,218,198,205,267,252,236,258,207,3310
9,Australia,Oceania,Australia and New Zealand,Developed regions,702,639,484,317,317,319,...,909,875,1033,1018,1018,933,851,982,1121,23829
